In [1]:
import numpy as np
import pickle
import string
import re
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [2]:
# Load the saved model
loaded_model = tf.keras.models.load_model("models/PD_Bidirectional_model_updated.keras")
# Convert the model to JSON
model_json = loaded_model.to_json()
    
# Load the tokenizer
with open("models/PD_Bidirectional_tokenizer_updated.pkl", "rb") as file:
    loaded_tokenizer = pickle.load(file)

In [3]:
pattern = r'[^A-Za-z\s]'
clean_str = lambda s: re.sub(pattern, '', str(s)).lower()

def print_seq(seq,tokenizer):
    return tokenizer.sequences_to_texts([seq])[0]
    
def beam_search_decoder(model, input_sequence, tokenizer, beam_width, max_len, gen_len):
    # Initialize beam search
    sequences = [[[], 0.0]]
    
    # Iterate through each prediction step
    for _ in range(gen_len):
        all_candidates = []
        # Prepare inputs for batch prediction
        input_seqs = np.array([seq + [0] * (max_len - len(seq)) for seq, _ in sequences])
        input_seqs = pad_sequences(input_seqs, maxlen=max_len)
        # Getting Generating outputs of all the beams
        for i in range(len(input_seqs)):
            print(tokenizer.sequences_to_texts([input_seqs[i]])[0])
        print('----------------------------------------------------')
        # Predict for all candidates in a batch
        preds = model.predict([np.tile(input_sequence, (len(sequences), 1)), input_seqs],verbose = 0)
        # Generate candidates for each sequence
        for i, (seq, score) in enumerate(sequences):
            if len(seq) > 0:
                pred = preds[i][-1]
                top_scores_indices = np.argsort(pred)[-beam_width:]
                for index in top_scores_indices:
                    candidate = [seq + [index], score + np.log(pred[index])]
                    all_candidates.append(candidate)
            else:
                pred = model.predict([input_sequence, np.zeros((1, max_len))],verbose = 0)[0][-1]
                top_scores_indices = np.argsort(pred)[-beam_width:]

                for index in top_scores_indices:
                    candidate = [[index], score + np.log(pred[index])]
                    all_candidates.append(candidate)

        # Select top-k candidates
        ordered = sorted(all_candidates, key=lambda x: x[1], reverse=True)
        sequences = ordered[:beam_width]
    return sequences

def generate_response(input_text, model, tokenizer):
    input_seq = tokenizer.texts_to_sequences([clean_str(input_text)])
    max_len, gen_len, beam_width = 110,40,2
    input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')
#     print('User Tokenized:',tokenizer.sequences_to_texts(input_seq)[0])
    top_k_sequences = beam_search_decoder(model, input_seq, tokenizer, beam_width, max_len, gen_len)
    decoded_sequences = [(tokenizer.sequences_to_texts([sequence[0]])[0],sequence[1]) for sequence in top_k_sequences]
    responses, scores = [x[0] for x in decoded_sequences], [x[1] for x in decoded_sequences]
    response = random.choice(responses)
    response = responses[0]
    return response, responses, scores

In [4]:
# prompt = 'Hello, I am feeling anxious. please help me.'
prompt = input('User:')
while prompt != '':
    response,_,_= generate_response(prompt, loaded_model, loaded_tokenizer)
    print(f'BOT: {response}')
    prompt = input('User:')

User:Hello, I am feeling anxious. please help me.

----------------------------------------------------
early
uncomfortable
----------------------------------------------------
early decisions
uncomfortable can
----------------------------------------------------
early decisions can
uncomfortable can be
----------------------------------------------------
early decisions can be
uncomfortable can be a
----------------------------------------------------
early decisions can be a
early decisions can be challenging
----------------------------------------------------
early decisions can be challenging but
early decisions can be a natural
----------------------------------------------------
early decisions can be challenging but its
early decisions can be a natural response
----------------------------------------------------
early decisions can be a natural response to
early decisions can be challenging but its essential
----------------------------------------------------
early decisions 

early decisions can be challenging but its essential to remember that everyone is crucial for our wellbeing <end>
early decisions can be challenging but its essential to remember that everyone is a natural response to your feelings of guilt and regret <end>
----------------------------------------------------
early decisions can be challenging but its essential to remember that everyone is crucial for our wellbeing <end>
early decisions can be challenging but its essential to remember that everyone is a natural response to your feelings of guilt and regret <end>
----------------------------------------------------
early decisions can be challenging but its essential to remember that everyone is crucial for our wellbeing <end>
early decisions can be challenging but its essential to remember that everyone is a natural response to your feelings of guilt and regret <end>
----------------------------------------------------
BOT: early decisions can be challenging but its essential to rememb

speaking overthinking and imagining worstcase scenarios are common in managing anxiety well work on cognitivebehavioral techniques to help you stay present and reduce anxiety additionally well explore relaxation techniques
speaking overthinking and imagining worstcase scenarios are common in managing anxiety well work on cognitivebehavioral techniques to help you stay present and reduce anxiety additionally well explore relaxation exercises
----------------------------------------------------
speaking overthinking and imagining worstcase scenarios are common in managing anxiety well work on cognitivebehavioral techniques to help you stay present and reduce anxiety additionally well explore relaxation techniques and
speaking overthinking and imagining worstcase scenarios are common in managing anxiety well work on cognitivebehavioral techniques to help you stay present and reduce anxiety additionally well explore relaxation techniques to
-------------------------------------------------

judgment its common to feel that youve been communicating about your feelings of judgment
judgment its common to feel that youve been communicating about your feelings of loneliness
----------------------------------------------------
judgment its common to feel that youve been communicating about your feelings of loneliness and
judgment its common to feel that youve been communicating about your feelings of judgment and
----------------------------------------------------
judgment its common to feel that youve been communicating about your feelings of loneliness and isolation
judgment its common to feel that youve been communicating about your feelings of loneliness and remember
----------------------------------------------------
judgment its common to feel that youve been communicating about your feelings of loneliness and remember that
judgment its common to feel that youve been communicating about your feelings of loneliness and isolation have
-------------------------------------

BOT: judgment its common to feel that youve been communicating about your feelings of loneliness and remember that might be contributing to your anxiety <end>
User:
